In [1]:
import pandas as pd
import numpy as np
import copy
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
comments = pd.read_csv('comments_raw.csv')
comments.head()

,page,comment_id,user_id,date,comment
0,1,3504465,410384,"April 11, 2007 6:52PM",Hi Pat:You forgot the Chrysler Sebring
1,1,3515400,209396,"April 11, 2007 7:33PM",I'm sure some folks would appreciate having th...
2,1,3516719,457562,"April 12, 2007 6:51AM",You can try to revive this topic but without b...
3,1,3517791,410027,"April 12, 2007 8:43AM",Model vs. model is exactly what we're here for...
4,1,3518875,411850,"April 13, 2007 11:49AM",The Altima is my favorite of the bunch. It is ...


In [3]:
comments.shape

(7500, 5)

In [4]:
text = comments['comment'].str.cat(sep=' ')
text

'Hi Pat:You forgot the Chrysler Sebring   I\'m sure some folks would appreciate having the Malibu included as well.  You can try to revive this topic but without being able to discuss (however heated the discussion gets..or not) manufacturer verses manufacturer or model verses model (as in making comparisons) or issues with a specific manufacturer there is little to talk about. I do agree about issues with other members/posters however in spite of being guilty of doing just that.  Model vs. model is exactly what we\'re here for! Manufacturer vs. manufacturer belongs elsewhere as I\'ve noted a number of times. There were many times we were turning the previous discussion into conversations that were/are already occurring in Auto News and they were happening to the exclusion of the vehicles themselves. That\'s what we need to avoid this time around.  The Altima is my favorite of the bunch. It is amongst the fastest and best handling of the group, is priced well, and has all the gadgets. 

In [5]:
# tokenize, lowercase and remove punctuation
words = nltk.word_tokenize(text)
words = [word.lower() for word in words if word.isalpha()]

In [6]:
# remove stopwords
stop_words = set(stopwords.words('english'))
final_tokens = []

for each in words:
    if each not in stop_words:
        final_tokens.append(each)
        
len(final_tokens)

335185

In [7]:
# lemmatize
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in final_tokens]
print(len(lemmatized_words))

string_words = ''
for el in lemmatized_words: 
    string_words += el + ' '

335185


In [8]:
# vectorizer 
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([string_words])
vectorizer.get_feature_names()

['aa',
 'aaa',
 'aaaargh',
 'aaargh',
 'aah',
 'aai',
 'aarp',
 'ab',
 'abandon',
 'abandoned',
 'aberage',
 'aberration',
 'abililties',
 'ability',
 'able',
 'abnormal',
 'abnormally',
 'abounds',
 'abouth',
 'abreast',
 'abrupt',
 'absence',
 'absolute',
 'absolutely',
 'absorb',
 'absorbed',
 'absorbing',
 'absorbs',
 'absorption',
 'absoultley',
 'absurd',
 'absurdly',
 'abundant',
 'abundantly',
 'abuse',
 'abused',
 'abusing',
 'abusive',
 'abysmal',
 'ac',
 'acadia',
 'acc',
 'acccord',
 'accelerate',
 'accelerated',
 'accelerates',
 'accelerating',
 'acceleration',
 'accelerative',
 'accelerator',
 'accent',
 'accented',
 'accentuate',
 'accentuated',
 'accentuates',
 'accept',
 'acceptable',
 'acceptably',
 'acceptance',
 'accepted',
 'accepting',
 'accepts',
 'accerating',
 'accesories',
 'access',
 'accessing',
 'accessory',
 'accident',
 'accidental',
 'accidentally',
 'accidentssince',
 'acclaim',
 'accleration',
 'accolade',
 'accommodate',
 'accommodated',
 'accommodati

In [9]:
# countvectorizer
cvectorizer = CountVectorizer()
X = cvectorizer.fit_transform([string_words])
column1 = cvectorizer.get_feature_names()
column2 = X.toarray()

df1 = pd.DataFrame(column2[0], columns=['Count'], index=column1)
df1.sort_values(by=['Count'], ascending=False, inplace=True)
df1

,Count
car,7674
accord,4566
would,3298
like,3006
one,2592
...,...
jest,1
jersey,1
jepordy,1
jeopardizes,1


In [10]:
df1.reset_index(inplace=True)
df1.rename(columns={"index": "Word"}, inplace=True)
df1

,Word,Count
0,car,7674
1,accord,4566
2,would,3298
3,like,3006
4,one,2592
...,...,...
13882,jest,1
13883,jersey,1
13884,jepordy,1
13885,jeopardizes,1


In [11]:
df1['POS'] = df1['Word'].apply(lambda x: nltk.pos_tag([x])[0][1])
df1

,Word,Count,POS
0,car,7674,NN
1,accord,4566,NN
2,would,3298,MD
3,like,3006,IN
4,one,2592,CD
...,...,...,...
13882,jest,1,NN
13883,jersey,1,NN
13884,jepordy,1,NN
13885,jeopardizes,1,NNS


In [12]:
# total count in df1
print(sum(df1['Count']))

333871


In [13]:
# export to csv
df1.to_csv('word_frequency.csv')

In [14]:
# import car and model list
cars = pd.read_csv('models.csv', header=None)
cars.rename(columns={0: 'car', 1: 'model'}, inplace=True)
cars['car'] = cars['car'].str.lower()
cars['model'] = cars['model'].str.lower()
cars

,car,model
0,acura,integra
1,acura,legend
2,acura,vigor
3,acura,rlx
4,acura,ilx
...,...,...
523,volvo,xc90
524,volvo,s60
525,volvo,s80
526,volvo,v60


In [15]:
# extract car brands and models
car_list = []
model_list = []

for w in df1['Word']:
    for c in cars.car.unique():
        if w == c:
            car_list.append(w)
    for m in cars.model.unique():
        if w == m:
            model_list.append(w)
        
print(car_list)
print(model_list)

['car', 'honda', 'ford', 'sedan', 'toyota', 'mazda', 'problem', 'seat', 'hyundai', 'nissan', 'chrysler', 'saturn', 'bmw', 'subaru', 'dodge', 'acura', 'kia', 'volvo', 'buick', 'audi', 'mitsubishi', 'mercedes', 'lincoln', 'cadillac', 'pontiac', 'mercury', 'chevrolet', 'infiniti', 'suzuki', 'volkswagen']
['accord', 'fusion', 'camry', 'sonata', 'altima', 'aura', 'malibu', 'civic', 'legacy', 'taurus', 'lx', 'city', 'impala', 'focus', 'mustang', 'vw', 'milan', 'sebring', 'comfort', 'elantra', 'chevy', 'hondas', 'camrys', 'passat', 'avalon', 'corolla', 'azera', 'sentra', 'tsx', 'odyssey', 'explorer', 'jetta', 'versa', 'prius', 'galant', 'escape', 'lacrosse', 'pilot', 'santa', 'mercury', 'buicks', 'tundra', 'altimas', 'leaf', 'rdx', 'lancer', 'mazdas', 'golf', 'avenger', 'corvette', 'mkz', 'viper', 'hyundais', 'accent', 'crv', 'outback', 'stellar', 'iq', 'charger', 'camaro', 'cruiser', 'forester', 'ridgeline', 'sienna', 'bmws', 'click', 'spark', 'century', 'rabbit', 'malibus', 'prelude', 'chry

In [16]:
# replace models with cars
df2 = df1.copy()
for m in model_list:
    mcar = cars.loc[cars['model'] == m, 'car'].iloc[0]
    df2 = df2.replace(m, mcar)
df2

,Word,Count,POS
0,car,7674,NN
1,honda,4566,NN
2,would,3298,MD
3,like,3006,IN
4,one,2592,CD
...,...,...,...
13882,jest,1,NN
13883,jersey,1,NN
13884,jepordy,1,NN
13885,jeopardizes,1,NNS


In [17]:
df2 = df2.groupby(['Word','POS'])['Count'].apply(np.sum).reset_index()

In [18]:
df2.sort_values(by=['Count'], ascending=False, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2

,Word,POS,Count
0,car,NN,7674
1,honda,NN,7170
2,ford,NN,3975
3,would,MD,3298
4,like,IN,3006
...,...,...,...
13733,jeff,NN,1
13734,jeanninejeannine,NN,1
13735,jealous,JJ,1
13736,jdpowers,NNS,1


In [19]:
# ensure the count is the same between df1 and df2
print(sum(df2['Count']))

333871


In [20]:
# export to csv
df2.to_csv('word_frequency2.csv')

### Top 5 Cars in Frequency
1. Honda
2. Ford
3. Toyota
4. Hyundai
5. Nissan